# load the dataset

using the titanic dataset, downloaded from [data.world](https://data.world/nrippner/titanic-disaster-dataset)

description:

- survival - Survival (0 = No; 1 = Yes)
- class - Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
- name - Name
- sex - Sex
- age - Age
- sibsp - Number of Siblings/Spouses Aboard
- parch - Number of Parents/Children Aboard
- ticket - Ticket Number
- fare - Passenger Fare
- cabin - Cabin
- embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
- boat - Lifeboat (if survived)
- body - Body number (if did not survive and body was recovered)

In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder
from matplotlib import pyplot as plt
from impyute.imputation.cs import mice


pd.set_option('display.width', 5000, 'display.max_columns', 100)

In [2]:
with open('./data/titanic.csv', mode='r') as file:
    df = pd.read_csv(file)

> first look at the data

In [3]:
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [4]:
df.describe()

,pclass,survived,age,sibsp,parch,fare,body
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,121.000000
mean,2.294882,0.381971,29.881135,0.498854,0.385027,33.295479,160.809917
std,0.837836,0.486055,14.413500,1.041658,0.865560,51.758668,97.696922
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800,72.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,155.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000,256.000000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200,328.000000


In [5]:
df.shape

(1310, 14)

1310 observations, 14 attributes

## data conversion

In [6]:
df.dtypes

pclass       float64
survived     float64
name          object
sex           object
age          float64
sibsp        float64
parch        float64
ticket        object
fare         float64
cabin         object
embarked      object
boat          object
body         float64
home.dest     object
dtype: object

In [7]:
for i in ['pclass', 'sex', 'embarked', 'boat']:
    df[i] = df[i].astype('category')

> droppint the name, cabin and home port since they offer no value to the prediction (and may lead the model to overfit on the person level)


In [8]:
df.loc[(~df['body'].isna()) & (df['survived'])].shape

(0, 14)

In [9]:
df.loc[(df['boat'].isna()) & (df['survived'])].shape

(23, 14)


> also dropping the boat, because it can only be known if a passenger survived so would be considered 'unfair' (even though apprently 24 passengers survived without one...)

> body number is also only given to people who didn't survive

In [10]:
df.drop(columns=['cabin', 'home.dest', 'boat', 'body', 'ticket'], inplace=True)

show value counts for factor columns

In [11]:
df['survived'] = df['survived'].astype(bool)

## title column & derivations (unfinished)

In [12]:
df['title'] = df['name'].str.extract('([A-Za-z]+)\.', expand=True)

In [13]:
df['title'].value_counts()

Mr          757
Miss        260
Mrs         197
Master       61
Dr            8
Rev           8
Col           4
Ms            2
Major         2
Mlle          2
Lady          1
Countess      1
Dona          1
Don           1
Capt          1
Mme           1
Sir           1
Jonkheer      1
Name: title, dtype: int64

> some titles are synonyms, some contain information about **marital status** or **social status**

### todo - extract military status, nationality, social status from titles

In [14]:
raise AssertionError

AssertionError: 

In [ ]:
title_synonyms = {
    'Mlle': 'Miss', 
    'Major': 'Mr', 
    'Col': 'Mr', 
    'Sir': 'Mr',
    'Don': 'Mr', 
    'Mme': 'Mrs', 
    'Jonkheer': 'Mr', 
    'Lady': 'Mrs',
    'Capt': 'Mr', 
    'Countess': 'Mrs', 
    'Ms': 'Miss', 
    'Dona': 'Mrs'}
df.replace({'title': title_synonyms}, inplace=True)

In [ ]:
df.head()

In [ ]:
raise AssertionError

In [ ]:
for i in filter(lambda x: df[x].dtype.name == 'category', df.columns):
    print(i)
    print(df[i].value_counts())

In [ ]:
df.head()

In [ ]:
df.isna().mean().sort_values(ascending=False)

In [ ]:
df.loc[df['sex'].isna()]

> note - one passenger (number 1309) has no data available at all. --> drop

In [ ]:
df.drop(labels=1309, inplace=True)

## dealing with missing values

- are values missing at random - check by pairplot
- values that are missing at random, impute with median
- values not missing at random, impute using [MICE](https://www.researchgate.net/publication/44203418_MICE_Multivariate_Imputation_by_Chained_Equations_in_R#:~:text=Multivariate%20Imputation%20by%20Chained%20Equations%20(MICE)%20is%20the%20name%20of,Conditional%20Speci%20cation%20(FCS).&text=0%2C%20the%20analysis%20of%20imputed,pooling%20works%20is%20substantially%20extended.)

### age

In [ ]:
df['age_missing'] = np.where(df['age'].isna(), True, False)
sns.pairplot(df, hue='age_missing', kind='kde')

> missing age does not seem to be correlated with anything **except** sbsp (number of siblings and spouses) and parch (number of parents and children) - those may have been young children

In [ ]:
df.loc[df['age'].isna(), 'sibsp'].value_counts()

In [ ]:
df.drop(columns=['age_missing'], inplace=True)

> correction - age is mostly missing with people who had no parents/siblings (and mostly did not survive - **not random -> use MICE imputation**

In [ ]:
df.head()

In [ ]:
df['sex'] = np.where(df['sex'] == 'female', 1, 0)

> one-hot encoding the embarkation columns (drop 1?)

In [ ]:
df = pd.concat([df, pd.get_dummies(df['embarked'], prefix='embarked')], axis=1)
df.drop(columns=['embarked', 'embarked_S'], inplace=True)

In [ ]:
df.drop(columns=['survived']).head()

In [ ]:
df.head()

In [ ]:
df['survived'] = df['survived'].astype(int)

In [ ]:
imputed_df = pd.DataFrame(data=mice(df.values), columns=df.columns)

In [ ]:
imputed_df

In [ ]:
#imputer = IterativeImputer(random_state=42)
#imputer.fit_transform(df)

In [ ]:
plt.figure(figsize=(20, 20))
sns.heatmap(df.corr(), cmap='Spectral')

In [ ]:
sns.pairplot(df, hue='survived')

## basic classifiers using sklearn